<a href="https://colab.research.google.com/github/Amirosein/MilkywayFormation/blob/main/Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Classifier Function

# Pre

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import io, requests, zipfile
from zipfile import ZipFile
import seaborn as sns

!pip install tsfresh
from tsfresh import extract_features

!pip install pyunpack
!pip install patool
from pyunpack import Archive

import warnings
warnings.filterwarnings('ignore')


# plt.rcParams.update({'font.size': 20, 'figsize':(8,6)})
SMALL_SIZE = 12
MEDIUM_SIZE = 15
BIGGER_SIZE = 18

plt.rc('font', size=BIGGER_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=MEDIUM_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=BIGGER_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=MEDIUM_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

plt.rc('axes', linewidth = 5 )
plt.rc('lines', linewidth = 5 )

plt.rc('lines', markersize = 5 )

plt.rc('figure', figsize=(8,4) )         # Image size

# load model

In [ ]:
import pickle

a_file = open("/content/finalized_model.sav", "rb")
model = pickle.load(a_file)
a_file.close()

b_file = open("/content/ktfcp.pkl", "rb")
kind_to_fc_parameters = pickle.load(b_file)
b_file.close()

from sklearn import preprocessing
encoder = preprocessing.LabelEncoder()
encoder.classes_ = np.load('/content/classes.npy', allow_pickle=True)

In [ ]:
from pathlib import Path
import csv
from pathlib import Path

f = open("/content/draft.csv", "x")

# function
def data2csv(path, var, sn):
  files = Path(path).rglob("*.dat")

  i = 0

  for file in files:
    datContent = [i.strip().split() for i in open(file).readlines()]
    id = sn +'-'+ Path(file).stem
    
    # write it as a new CSV file
    with open("/content/draft.csv", "w") as f:
      writer = csv.writer(f)
      writer.writerows(datContent)

    df = pd.read_csv("/content/draft.csv", names=['time','value','e']) 
    df = df[['time','value']]
    df['id'] = id
    df['kind'] = var

    if i==0:
      data = df
    else:
      data = pd.concat([data,df], axis=0)

    i = i+1
    #print(i)

  return data

In [ ]:
def feat_ex(X):
  m = extract_features(X, column_id="id", column_sort="time", n_jobs= 20,
                       column_kind="kind", column_value="value",
                       kind_to_fc_parameters = kind_to_fc_parameters)
  return m

In [ ]:
def dat2feat(urlI,urlV,sn):
  mI = data2csv(urlI, 'I',sn)
  mV = data2csv(urlV, 'V',sn)
  m = pd.concat([mI,mV])
  
  f = feat_ex(m)
  return f

# function

In [ ]:
def classifier(url_I,url_V,sn):
  # sn = survey name
  
  features = dat2feat(url_I,url_V,sn)
  y = model.predict(features)
  encoded_y = encoder.inverse_transform(y)
  return encoded_y